In [28]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np
import pandas as pd
import pickle
from timeit import default_timer as timer
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
import nltk as nltk

import os
import cProfile, pstats, io

# Enabling eager execution 

In [7]:
tf.enable_eager_execution()

# Load Google vectors

In [8]:
UNK = '</s>'

dirname = os.getcwd()
googlePath = os.path.join(dirname, 'datasets/')
outfile = googlePath +'google_word_corpus.pic'

with open(outfile, 'rb') as pickle_file:    
    googleCorpus, google_corpus_word_to_int, google_corpus_int_to_word = pickle.load(pickle_file)
googleSet = pd.read_csv(googlePath+'GoogleNews-vectors-negative10.txt', sep=' ', header=None)
print(googleSet.shape)
print(googleSet.head())

googleWords = googleSet.iloc[:,0:1]
googleVectors = googleSet.iloc[:,1:]

(1965717, 11)
     0         1         2         3         4         5         6         7   \
0  </s>  0.001129 -0.000896  0.000319  0.001534  0.001106 -0.001404 -0.000031   
1    ##  0.168945 -0.062988 -0.000265  0.063477  0.087402 -0.030029  0.001450   
2  ####  0.107910  0.094238 -0.018555 -0.018799 -0.007111 -0.139648 -0.046631   
3     $  0.113770 -0.117676  0.064941  0.132812  0.054932 -0.099121  0.071289   
4   ###  0.149414 -0.191406  0.044189  0.104980  0.016846 -0.135742 -0.085449   

         8         9         10  
0 -0.000420 -0.000576  0.001076  
1 -0.077637  0.199219 -0.001770  
2 -0.063965  0.098145  0.135742  
3 -0.013306  0.271484 -0.083984  
4  0.074219  0.088867 -0.044922  


In [9]:
#UNK = '<unk>'

#glovePath = '/media/kandy/hdd/master-thesis/datasets/'
#outfile = glovePath +'glove_word_corpus.pic'

#with open(outfile, 'rb') as pickle_file:    
#    gloveCorpus, glove_corpus_word_to_int, glove_corpus_int_to_word = pickle.load(pickle_file)
#gloveSet = pd.read_csv(glovePath+'glove.42B.10d.txt', sep=' ', header=None)
#print(gloveSet.shape)
#print(gloveSet.head())

#gloveWords = gloveSet.iloc[:,0:1]
#gloveVectors = gloveSet.iloc[:,1:]

# Load training and test dataset

In [10]:
treeDataframe = pd.read_csv(googlePath+'constituency-parsing-data-all-UNK.csv', sep=' ', header=None )
treeDataframe.columns =['sentence', 'tree']
treeDataframe['tree'] = treeDataframe['tree'].apply(nltk.Tree.fromstring)

treeDataframe.head()

,sentence,tree
0,pierre </s> </s> </s> years old </s> will join...,"[[[(</s> (</s> pierre) (</s> vinken)), (</s> <..."
1,</s> </s> is chairman </s> </s> </s> </s> the ...,"[[[(</s> mr</s>), (</s> vinken)], [(</s> is), ..."
2,</s> </s> </s> </s> years old </s> former chai...,"[[[(</s> (</s> rudolph) (</s> agnew)), (</s> <..."
3,</s> form </s> asbestos once used * * </s> mak...,[[[(</s>\n (</s>\n (</s> (</s> a) (</s> fo...
4,the asbestos fiber </s> </s> </s> is unusually...,[[[(</s>\n (</s> (</s> the) (</s> asbestos) (...


In [11]:
def convert_imdb_corpus_into_int(sentence):
    words = sentence.split()
    words_to_num = [google_corpus_word_to_int[word] for word in words]
    return words_to_num

treeDataframe_num = treeDataframe.copy()
treeDataframe_num['sentence'] = treeDataframe_num['sentence'].apply(convert_imdb_corpus_into_int)
treeDataframe_num.head()

,sentence,tree
0,"[353287, 920150, 920150, 920150, 1131873, 1851...","[[[(</s> (</s> pierre) (</s> vinken)), (</s> <..."
1,"[920150, 920150, 1025196, 261666, 920150, 9201...","[[[(</s> mr</s>), (</s> vinken)], [(</s> is), ..."
2,"[920150, 920150, 920150, 920150, 1131873, 1851...","[[[(</s> (</s> rudolph) (</s> agnew)), (</s> <..."
3,"[920150, 1751421, 920150, 151538, 26198, 83139...",[[[(</s>\n (</s>\n (</s> (</s> a) (</s> fo...
4,"[1323513, 151538, 1349057, 920150, 920150, 920...",[[[(</s>\n (</s> (</s> the) (</s> asbestos) (...


In [44]:
treeDF_train, treeDF_test = sklearn.model_selection.train_test_split(treeDataframe_num, test_size=0.9)
print(treeDF_train.shape)
print(treeDF_test.shape)

(4920, 2)
(44288, 2)


# Model and the Parameters

In [45]:
STATE_SIZE = 10
embeddings = tfe.Variable(name='embeddings', validate_shape= googleVectors.shape, 
                          initial_value=googleVectors.values, 
                          dtype=tf.float32, trainable=False)
w = tfe.Variable(name='w', validate_shape=(2*googleVectors.shape[1], STATE_SIZE), 
                 initial_value=0.01 * tf.random_normal(shape=(2*googleVectors.shape[1], STATE_SIZE)),
                 dtype=tf.float32)
b = tfe.Variable(name='b', validate_shape=(1, STATE_SIZE),
                 initial_value=0.01 * tf.random_normal(shape=(1, STATE_SIZE)),
                 dtype=tf.float32)

w_score = tfe.Variable(name='w_score', validate_shape=(STATE_SIZE, 1), 
                 initial_value=0.01 * tf.random_normal(shape=(STATE_SIZE, 1)),
                 dtype=tf.float32)
b_score = tfe.Variable(name='b_score', validate_shape=(1, 1),
                 initial_value=0.01 * tf.random_normal(shape=(1, 1)),
                 dtype=tf.float32)

#print(w)
#print(b)
#print(w_score)
#print(b_score)

In [46]:
print(embeddings.shape)
lookup = tf.nn.embedding_lookup(embeddings, google_corpus_word_to_int['</s>'])
lookup = tf.reshape(lookup, shape=(1, lookup.shape[0]))
print(lookup)

(1965717, 10)
tf.Tensor(
[[ 0.09570312 -0.03881836  0.05883789  0.03930664 -0.01300049 -0.1484375
   0.07568359 -0.13378906  0.2890625   0.00370789]], shape=(1, 10), dtype=float32)


In [47]:
def compute_score_for_tree(tree, weights, embeddings, scores):
    if(type(tree[0]) == type('a string')):
        #print(tree.label() + ' : ' + tree[0])
        word = tree[0].lower()
        if(word not in google_corpus_word_to_int):
            #print('missing word:',word)
            word = '</s>'
        word_vector = tf.nn.embedding_lookup(embeddings, google_corpus_word_to_int[word])
        word_vector = tf.reshape(word_vector, shape=(1, word_vector.shape[0]))
        return word_vector
        #print('depth is reached !!!')
        #return

    #for i in range(len(tree)):
    #    print('Inside tree : '+ tree[i].label())
    #    compute_score_for_tree(tree[i], weights, embeddings)


        
    left = compute_score_for_tree(tree[0], weights, embeddings, scores)
    if(len(tree) !=2):
        return left

    right = compute_score_for_tree(tree[1], weights, embeddings, scores)
    words_concat = tf.concat([left, right], axis=0)
    #print(words_concat.shape)
    #print(left.shape, right.shape)
    words_concat = tf.reshape(words_concat, shape=(1, left.shape[1] + right.shape[1]))
    #print(words_concat)
    z = tf.matmul(words_concat, weights[0]) + weights[1]
    state_vec = tf.tanh(z)
    
    #print(state_vec)
    score = tf.matmul(state_vec, weights[2]) + weights[3]
    scores.append(score)
    return state_vec
    
    
for j in range(treeDF_train.shape[0]):
    tree = treeDF_train.iat[j,1]
    print(tree.leaves())
    scores = []
    tree.chomsky_normal_form()
    compute_score_for_tree(tree[0], [w,b,w_score,b_score], embeddings, scores)
    #print(sum(scores))
    total_score = tf.reduce_sum(tf.stack(scores))
    print(total_score)
    break
    


['past', 'colombian', 'government', 'tolerance', 'of', 'the', '</s>', 'narcotraficantes', '</s>', 'was', 'due', 'to', 'the', 'drug', 'lords', "'", 'history', 'of', '*', 'wiping', 'out', 'leftists', 'in', 'the', 'hinterlands', '</s>']
tf.Tensor(0.15503682, shape=(), dtype=float32)


In [48]:
def compare_trees(actual, predicted):
    actual_sub_trees = actual.subtrees()
    
    
    total_count = 0.0 #len(actual_sub_trees)
    match_count = 0.0
    
    for sub_tree in actual_sub_trees:
        #print('Sub Tree:', sub_tree)
        #print(predicted_sub_trees)
        predicted_sub_trees = predicted.subtrees()
        for temp_tree in predicted_sub_trees:
            #print(' ', temp_tree)
            if(sub_tree == temp_tree):
                match_count = match_count + 1.0
                break
        total_count = total_count + 1.0
        #break
    #print('Match:', match_count,' Total:', total_count)
    
    return [match_count/total_count, match_count, total_count]

#compare_trees(treeDF_train.iat[2,1], treeDF_train.iat[2,1])

t1 = nltk.Tree.fromstring("(S (NP I) (VP (V saw) (NP him)))")
sub_trees = t1.subtrees()
for sub_tree in sub_trees:
    print(sub_tree)

(S (NP I) (VP (V saw) (NP him)))
(NP I)
(VP (V saw) (NP him))
(V saw)
(NP him)


In [49]:
#t1 = nltk.Tree.fromstring("(S (NP I) (VP (V saw) (NP him)))")
#t2 = nltk.Tree.fromstring("(a (b I) (c (d saw) (e him)))")

#compare_trees(t1, t2)

In [50]:
pr = cProfile.Profile()
pr.enable()
# ... do something ...

In [51]:
start = timer()
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
epoch = 1

# history to maintain variables,  losses and gradients
variables_history = []
variables_history.append([np.copy(w.numpy()), np.copy(b.numpy()),np.copy(w_score.numpy()),np.copy(b_score.numpy())])

loss_history = []
grad_history = []

with tf.device("/cpu:0"):
    for i in range(epoch):
        # to record the history of tensor computations
        total_train_score = 0.0
        total_train_count = 0.0
        print('Epoch : ', i)
        with tf.GradientTape() as tape:
            # watching - is it necessary ???
            #tape.watch([w,b,w_score,b_score])
            for j in range(treeDF_train.shape[0]):
                # get the word vectors based on the word ids (word id for each word)
                words = tf.nn.embedding_lookup(embeddings, treeDF_train.iat[j,0])
                end = timer()
                #print('Time taken to lookup embeddings (seconds): ', end-start)
                #words matrix - unstack
                words_unstack = tf.unstack(words)
                words_len = len(words_unstack)
                pred_score_list = []
                total_loss_list = []
                predicted_tree = [nltk.Tree(UNK,[google_corpus_int_to_word[index]]) for index in treeDF_train.iat[j,0]]

                state_vec_list = []
                score_list = []

                start_k = 0
                stop_k = words_len - 1 
                #loop until all the words are merged together
                while(words_len > 1):
                    #compute scores for the list of word combinations
                    # for each word combination compute the score of it

                    scores = np.zeros(shape=(words_len-1, 1))
                    for k in range(start_k, stop_k):
                        words_concat = tf.concat([words_unstack[k], words_unstack[k+1]], axis=0)
                        #reshape the tensor to be a matrix with 1 row rather than vector
                        words_concat = tf.reshape(words_concat, shape=(1, words_concat.shape[0]))
                        # matrix computation and activation
                        z = tf.matmul(words_concat, w) + b
                        state_vec = tf.tanh(z)
                        state_vec_list.append(state_vec)
                        score = tf.matmul(state_vec, w_score) + b_score
                        score_list.append(score)
                        scores[k] = score

                    end = timer()
                    #print('Time taken to calculate all subsequent word combinations (seconds): ', end-start)

                    #compare the scores and pick the maximum one. 
                    max_score_index = np.argmax(scores) 
                    pred_score_list.append(scores[max_score_index])

                    # remove the words which is used to combine and replace with combined state vector
                    words_unstack.pop(max_score_index+1)
                    words_unstack.pop(max_score_index)
                    # statevector needs to be reshaped as matrix to update
                    state_vec_vector = tf.reshape(state_vec, shape = [state_vec.shape[1]])
                    words_unstack.insert(max_score_index, state_vec_vector)
                    words_len = len(words_unstack)

                    right_tree = predicted_tree.pop(max_score_index+1)
                    left_tree = predicted_tree.pop(max_score_index)
                    predicted_tree.insert(max_score_index, nltk.Tree(UNK, [left_tree, right_tree]))

                    start_k = max(0, max_score_index - 1)
                    stop_k = min(max_score_index+2, words_len-1)

                    #print([max_score_index, start_k, stop_k, words_len])

                    end = timer()
                    #print('Time taken to make one decision (seconds): ', end-start)

                end = timer()
                if (j%10 == 0):
                    print(j,end-start)
                #print('Time taken to forward loop (seconds): ', end-start)
                # get the actual tree - convert it to chomsky normal form, and compute the score
                act_score_list = []
                tree = treeDF_train.iat[j,1]
                tree.chomsky_normal_form()
                compute_score_for_tree(tree[0], [w,b,w_score,b_score], embeddings, act_score_list)
                end = timer()
                #print('Time taken to chomsky calculation (seconds): ', end-start)

                # compute the total actual and predicted score. use the loss function as absolute difference
                # the above is done for each training data and the loss are accumulated
                total_act_score = tf.reduce_sum(tf.stack(act_score_list))
                total_pred_score = tf.reduce_sum(tf.stack(pred_score_list))
                loss = tf.losses.absolute_difference(total_act_score,  total_pred_score)
                #total_loss_list.append(loss)
                #loss = tf.losses.sigmoid_cross_entropy(tf.constant(imdb_train.iat[j,1], shape=(1,1)), y_predict)

                total_train_score = total_train_score + compare_trees(tree[0], predicted_tree[0])[0]
                total_train_count = total_train_count + 1

            #compute the average losses accompanying all training data
            # compute the gradients and apply them on variables
            
            #total_loss = tf.reduce_mean(tf.stack(total_loss_list))
            #grads = tape.gradient(total_loss, [w,b,w_score,b_score])

            #print(w[0])

            #grad_op = optimizer.apply_gradients(zip(grads, [w,b,w_score,b_score]), 
            #                          global_step=tf.train.get_or_create_global_step())

            #print(w[0])
            #maintain the history of variables, losses and gradients

            #variables_history.append([np.copy(w.numpy()), np.copy(b.numpy()),np.copy(w_score.numpy()),np.copy(b_score.numpy())])
            #loss_history.append(np.copy(total_loss.numpy()))
            #grad_history.append([np.copy(grad.numpy()) for grad in grads])

            #print(tf.train.get_or_create_global_step().numpy(),total_loss.numpy())
            print('Training Accuracy : ', total_train_score/total_train_count)


    end = timer()
    print('Time taken to execute (seconds): ', end-start)

Epoch :  0
0 0.39154541000607423
10 1.1685107520024758
20 1.7845930930052418
30 2.578968147005071
40 2.9509501150023425
50 3.831273144998704
60 4.775569062010618
70 5.677500464007608
80 6.605780325000524
90 7.156542426004307
100 7.922992293999414
110 9.377924365006038
120 10.307860859000357
130 11.255826689011883
140 11.889361673005624
150 13.067481236997992
160 13.792607548006345
170 14.41770195600111
180 15.084705348999705
190 15.893446766000125
200 16.766056049003964
210 17.43914555000083
220 18.283094276004704
230 19.102967958009685
240 19.95888465200551
250 21.14320104800572
260 22.01998413700494
270 23.03194398400956
280 23.69726954700309
290 24.84750752900436
300 26.051860745006707
310 28.119792203011457
320 29.072694411006523
330 30.565129431008245
340 31.745763184007956
350 33.010958680999465
360 33.804098742009955
370 34.835903637009324
380 35.63980575199821
390 37.10869682900375
400 38.052061203998164
410 38.92684785400343
420 40.726808969004196
430 41.57681575400056
440 42.

3560 683.4424709160085
3570 686.1420135720109
3580 687.8627711470035
3590 689.1804756250058
3600 690.9271442010067
3610 692.7607286390121
3620 718.5336392590107
3630 719.872133953002
3640 794.801805731011
3650 820.9686476000061
3660 822.6011585259985
3670 824.199165486003
3680 825.9910917860107
3690 828.0922592850111
3700 830.0584985120076
3710 831.6783189490088
3720 833.2818148630031
3730 835.4046655550046
3740 836.9024874710012
3750 838.4368119959981
3760 840.151695737004
3770 842.0210794140003
3780 843.5175880780007
3790 844.7343428590102
3800 846.8559865230054
3810 848.3988948100014
3820 849.7201600300032
3830 851.603295961002
3840 853.7669683720014
3850 855.1756073580036
3860 857.0875208170037
3870 858.489119646998
3880 859.7597700830083
3890 861.361406246011
3900 863.3104898809979
3910 864.9014222380065
3920 866.5340819970006
3930 868.216937145
3940 869.9515759930073
3950 871.785807763008
3960 872.8438028590026
3970 874.623745615012
3980 876.2634251010022
3990 878.3551019050064
4

In [52]:
pr.disable()
s = io.StringIO()
sortby = 'cumulative'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())


         421564464 function calls (240550985 primitive calls) in 1413.941 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        7    0.000    0.000 1413.939  201.991 /home/kandy/anaconda3/envs/nlp-master-thesis/lib/python3.6/asyncio/base_events.py:1355(_run_once)
      8/5    0.000    0.000 1413.490  282.698 /home/kandy/anaconda3/envs/nlp-master-thesis/lib/python3.6/site-packages/tornado/gen.py:1115(run)
     12/7    0.000    0.000 1413.490  201.927 {method 'send' of 'generator' objects}
       20    0.000    0.000 1413.489   70.674 /home/kandy/anaconda3/envs/nlp-master-thesis/lib/python3.6/asyncio/events.py:143(_run)
     12/8    0.000    0.000 1413.489  176.686 /home/kandy/anaconda3/envs/nlp-master-thesis/lib/python3.6/site-packages/tornado/stack_context.py:296(null_wrapper)
        7    0.000    0.000 1413.488  201.927 /home/kandy/anaconda3/envs/nlp-master-thesis/lib/python3.6/site-packages/tornado/ioloop.py:752(_run

In [ ]:
tree[0]

In [ ]:
predicted_tree[0]

In [ ]:
compare_trees(tree[0], predicted_tree[0])

In [ ]:
' '.join(tree[0].leaves())

* Improve parallelization through batch processing
* Implement CMY algorithm to reuse the computed values
* Preprocessing of tree (binary children, UNK words)

* check for this tree ('morgenzon has long been a special domain of afrikanerdom </s>')
* still missing word in compute_score_for_tree function. why? All preprocessign are good
* Other hyper parameters

* GPU usage

* Increase no. of units
* BEAM search
* Involve the classification of subcategories
* Assign different weights for different subcategories (atleast broader)